<a href="https://colab.research.google.com/github/donaldziff/kgqa-ucb-210/blob/main/training/summarization/AskWiki_Flow_Inception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Askwiki Simulation Notebook 

1. This notebooks was done in inception days of askwiki 

2. To be used as huddle notebook to explore ideas and potential flows with examples 

Author : shrinivasbjoshi@berkeley.edu

# Step 1 Scenario users asks "what are diseases that can cause liver infection" in natural language ....




# Step 2 Our 1st model  converts this NL question into a sparql query which is 

"""SELECT DISTINCT ?sbj ?sbj_label WHERE { ?sbj wdt:P31 wd:Q18123741 . ?sbj wdt:P689 wd:Q9368 . ?sbj rdfs:label ?sbj_label . FILTER(STRSTARTS(lcase(?sbj_label), 'h')) . FILTER (lang(?sbj_label) = 'en') } LIMIT 25"""

# Step 3 we run this query against the wikidata end point and get results answer from wikidata is 3 diseases Hepatatis a b and c... you can run the query ..below is the code for it using python

In [ ]:
!pip install sparqlwrapper
import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

#query = """SELECT ?answer WHERE { wd:Q42807 wdt:P190 ?answer . ?answer wdt:P1792 wd:Q7118067}"""
query = """SELECT DISTINCT ?sbj ?sbj_label WHERE { ?sbj wdt:P31 wd:Q18123741 . ?sbj wdt:P689 wd:Q9368 . ?sbj rdfs:label ?sbj_label . FILTER(STRSTARTS(lcase(?sbj_label), 'h')) . FILTER (lang(?sbj_label) = 'en') } LIMIT 25"""

def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


results = get_results(endpoint_url, query)

for result in results["results"]["bindings"]:
    print(result)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.3/500.3 KB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 KB 5.6 MB/s eta 0:00:00
{'sbj': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q6853'}, 'sbj_label': {'xml:lang': 'en', 'type': 'literal', 'value': 'hepatitis B'}}
{'sbj': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q157661'}, 'sbj_label': {'xml:lang': 'en', 'type': 'literal', 'value': 'hepatitis A'}}
{'sbj': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q154869'}, 'sbj_label': {'xml:lang': 'en', 'type': 'literal', 'value': 'hepatitis C'}}


# Step 4 is we have to assemble contents from these wikidata pages and generate a summarized response for our user .... which means we should be able to get information not just about A B C and not just the values A B C 

# Now let us deep dive into this step 4

## we need to extract information from the pages for the entities of A B and C i.e http://www.wikidata.org/entity/Q6853 ,http://www.wikidata.org/entity/Q157661 , 'http://www.wikidata.org/entity/Q154869'




In [ ]:
!pip install Wikidata
from wikidata.client import Client
client = Client()  # doctest: +SKIP
# for testing purposes I am putting one argument for now
entity = client.get('Q6853', load=True)
entity

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


<wikidata.entity.Entity Q6853 'hepatitis B'>

### below is the structure of the object of the result 

In [ ]:
entity.data

{'pageid': 7986,
 'ns': 0,
 'title': 'Q6853',
 'lastrevid': 1826719920,
 'modified': '2023-02-05T08:59:00Z',
 'type': 'item',
 'id': 'Q6853',
 'labels': {'it': {'language': 'it', 'value': 'epatite B'},
  'en': {'language': 'en', 'value': 'hepatitis B'},
  'de': {'language': 'de', 'value': 'Hepatitis B'},
  'ar': {'language': 'ar', 'value': 'التهاب الكبد ب'},
  'as': {'language': 'as', 'value': 'হেপাটাইটিছ বি'},
  'az': {'language': 'az', 'value': 'Hepatit B'},
  'bg': {'language': 'bg', 'value': 'Хепатит B'},
  'ca': {'language': 'ca', 'value': 'hepatitis B'},
  'cs': {'language': 'cs', 'value': 'hepatitida B'},
  'dv': {'language': 'dv', 'value': 'ހެޕަޓައިޓިސް ބީ'},
  'el': {'language': 'el', 'value': 'ηπατίτιδα Β'},
  'es': {'language': 'es', 'value': 'hepatitis B'},
  'et': {'language': 'et', 'value': 'B-hepatiit'},
  'fa': {'language': 'fa', 'value': 'هپاتیت ب'},
  'fr': {'language': 'fr', 'value': 'hépatite B'},
  'ga': {'language': 'ga', 'value': 'heipitíteas B'},
  'he': {'langu

# We need to figureout how to convert above into a textual information so that  any  model can summarize it for generating answer to the user 

# Ideal model answer generate from any model would be 

The most common types of liver infection are hepatitis viruses, including:

Hepatitis A
Hepatitis B
Hepatitis C

Diseases in which your immune system attacks certain parts of your body (autoimmune) can affect your liver. Examples of autoimmune liver diseases include:

Autoimmune hepatitis
Primary biliary cholangitis
Primary sclerosing cholangitis
Genetics
An abnormal gene inherited from one or both of your parents can cause various substances to build up in your liver, resulting in liver damage. 